<a href="https://www.kaggle.com/code/taniamartynenko/eda-mll-s3e22tania?scriptVersionId=143884440" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Metrics to evaluate the model
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve,recall_score
from sklearn import tree


# For tuning the model
from sklearn.model_selection import GridSearchCV

# To ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier

## Brief description of data set columns:

- **id**: A unique identifier for each entry. (Integer)
- **surgery**: Information on whether the horse had surgery or not. (Categorical)
- **age**: Age of the horse. (Categorical)
- **hospital_number**: A unique identifier for the horse in the hospital. (Integer)
- **rectal_temp**: The rectal temperature of the horse. (Continuous, Float)
- **pulse**: The pulse rate of the horse. (Continuous, Float)
- **respiratory_rate**: The rate of respiration of the horse. (Continuous, Float)
- **temp_of_extremities**: The temperature of the horse's extremities, potentially indicating circulation issues. (Categorical)
- **peripheral_pulse**: Quality of the horse's peripheral pulse. (Categorical)
- **mucous_membrane**: The appearance of the horse's mucous membranes. (Categorical)
- **capillary_refill_time**: Time taken for color to return to an external capillary bed after pressure is applied. (Categorical)
- **pain**: The horse's level of pain. (Categorical)
- **peristalsis**: Gut motility observation. (Categorical)
- **abdominal_distention**: Severity of abdominal distention. (Categorical)
- **nasogastric_tube**: Type of nasogastric tube, if present. (Categorical)
- **nasogastric_reflux**: Presence and type of nasogastric reflux. (Categorical)
- **nasogastric_reflux_ph**: The pH of nasogastric reflux. (Continuous, Float)
- **rectal_exam_feces**: Results of the rectal examination for feces. (Categorical)
- **abdomen**: General appearance of the abdomen. (Categorical)
- **packed_cell_volume**: The volume percentage of red blood cells in blood. (Continuous, Float)
- **total_protein**: Total protein level in the blood. (Continuous, Float)
- **abdomo_appearance**: Appearance of the abdominal fluid. (Categorical)
- **abdomo_protein**: Protein content in abdominal fluid. (Continuous, Float)
- **surgical_lesion**: Indicates if a surgical lesion was present. (Categorical)
- **lesion_1**: Primary lesion type. (Integer)
- **lesion_2**: Secondary lesion type, if present. (Integer)
- **lesion_3**: Tertiary lesion type, if present. (Integer)
- **cp_data**: Clinical pathology data. (Categorical)
- **outcome**: The health outcome of the horse. (Categorical)
_________________________________________________________________________________

**Goal:** Use the above medical indicators to predict the health outcomes of horses.

### Loading Data:

In [ ]:
train_df = pd.read_csv('/kaggle/input/playground-series-s3e22/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s3e22/test.csv')
sample_submission = pd.read_csv('/kaggle/input/playground-series-s3e22/sample_submission.csv')

### Train Data:

In [ ]:
train_df.head()

In [ ]:
train_df.shape

In [ ]:
train_df.info()

Train Dataset has columns with numerical and categorical datatypes. Next will explore missinf value and check the datatype. 

### Checking dublicates and missing value: 

In [ ]:
# Let's check for duplicate values in the data
train_df.duplicated().sum()

Train data has 0 dublicate value.

In [ ]:
# Let's check for missing values in the data
round(train_df.isnull().sum() / train_df.isnull().count() * 100, 2)

**Observations:**
Traing Data has % of missing values in the next columns: 
- temp_of_extremities       3.16
- peripheral_pulse          4.86
- mucous_membrane           1.70
- capillary_refill_time     0.49
- pain                      3.56
- peristalsis               1.62
- abdominal_distention      1.86
- nasogastric_tube          6.48
- nasogastric_reflux        1.70
- rectal_exam_feces        15.38
- abdomen                  17.25
- abdomo_appearance         3.89


In [ ]:
#Checking Missing Values Pattern in Dataframe
import missingno as mn

null_cols = [i for i in train_df.columns if train_df[i].isnull().any()]
print(null_cols)
mn.matrix(train_df[null_cols])

Columns **rectal_exam_feces** has 15.38% of missing value and **abdomen** has 17.25% missing values and pattent showes that this misisng value is crossing entair dataset. Based on this observation needs to deal with this missing values for **rectal_exam_feces** and **abdomen** and do the same for test data. 

In [ ]:
# Let's check the number of unique values in each column
train_df.nunique()

In [ ]:
for i in train_df.describe(include=["object"]).columns:
    print("Unique values in", i, "are :")
    print(train_df[i].value_counts())
    print("*" * 50)

### Data Pre-processing - Train_df


In [ ]:
# ID column consists of uniques ID and hence will not add value to the modeling

saved_ids_train = train_df['id'].copy()

train_df.drop(columns="id", inplace=True)

In [ ]:
# ID column consists of uniques hospital_number and hence will not add value to the modeling
train_df.drop(columns="hospital_number", inplace=True)

### **Univariate analysis of numerical columns - Train_df**

In [ ]:
# Let's view the statistical summary of the numerical columns in the data
train_df.describe().T

### Univariant Analysist:

In [ ]:
train_df.dtypes

In [ ]:
num_cols = ['rectal_temp','pulse', 'respiratory_rate', 'nasogastric_reflux_ph',
            'packed_cell_volume','total_protein','abdomo_protein','lesion_1','lesion_2','lesion_3']
    

In [ ]:
# Set up the figure and axes
n = len(num_cols)

fig, axes = plt.subplots(n, 2, figsize=(12, 3*n))
#fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(16, 20))

# Loop through the numerical variables to plot histograms and box plots
for i, var in enumerate(num_cols):
    # Plot histogram
    axes[i, 0].hist(train_df[var], bins=30, alpha=0.7)
    axes[i, 0].set_title(f'{var} - Histogram')
    axes[i, 0].set_xlabel(var)
    axes[i, 0].set_ylabel('Frequency')
    
    # Plot box plot
    axes[i, 1].boxplot(train_df[var], vert=False)
    axes[i, 1].set_title(f'{var} - Box plot')

# Adjust the layout
plt.tight_layout()
plt.show()

### Correlation among variables Train Df

In [ ]:
correlation_matrix = train_df[num_cols].corr()
print(correlation_matrix)

### Ploting Correlation Matrix

In [ ]:
plt.figure(figsize=(8, 4))
sns.heatmap(train_df[num_cols].corr(), annot=True, vmin=-1, vmax=1, fmt=".2f", cmap="Spectral")
plt.show()

### **Observations:**

1. **Diagonal Elements**: 
   - The diagonal elements of the correlation matrix are always 1 because any variable is perfectly correlated with itself.
   
2. **Pulse and Packed Cell Volume**:
   - There's a moderate positive correlation between `pulse` and `packed_cell_volume` (\( r \approx 0.435 \)). This suggests that as the pulse rate increases, the volume percentage of red blood cells in the blood also tends to increase.
   
3. **Pulse and Respiratory Rate**:
   - `pulse` and `respiratory_rate` also show a moderate positive correlation (\( r \approx 0.395 \)). This means that horses with a higher pulse rate tend to have a higher respiratory rate.

4. **Nasogastric Reflux pH and Total Protein**:
   - There's a strong negative correlation between `nasogastric_reflux_ph` and `total_protein` (\( r \approx -0.583 \)). As the pH of nasogastric reflux increases, the total protein level in the blood tends to decrease.

5. **Nasogastric Reflux pH and Abdomo Protein**:
   - There's a positive correlation between `nasogastric_reflux_ph` and `abdomo_protein` (\( r \approx 0.433 \)). This means that as the pH of nasogastric reflux increases, the protein content in abdominal fluid also tends to increase.

6. **Lesion 2 and Lesion 3**:
   - There's a strong positive correlation between `lesion_2` and `lesion_3` (\( r \approx 0.644 \)). This suggests that the secondary and tertiary lesion types tend to occur together.

7. **Weak Correlations**:
   - Many of the correlations in the matrix are close to zero, indicating weak or no linear relationship between the paired variables.



## Handling missing values in train_df:

### -  Columns with Numeric Data (float64/int64):

For columns like **rectal_temp, pulse, respiratory_rate, nasogastric_reflux_ph, packed_cell_volume, total_protein, and abdomo_protein,**  can be use the median or mean of the column to fill in missing values.

In [ ]:
train_df['rectal_temp'].fillna(train_df['rectal_temp'].median(), inplace=True)
train_df['pulse'].fillna(train_df['pulse'].median(), inplace=True)
train_df['respiratory_rate'].fillna(train_df['respiratory_rate'].median(), inplace=True)
train_df['nasogastric_reflux_ph'].fillna(train_df['nasogastric_reflux_ph'].median(), inplace=True)
train_df['packed_cell_volume'].fillna(train_df['packed_cell_volume'].median(), inplace=True)
train_df['total_protein'].fillna(train_df['total_protein'].median(), inplace=True)
train_df['abdomo_protein'].fillna(train_df['abdomo_protein'].median(), inplace=True)


### -  Columns with Categorical Data (Object):

In [ ]:
train_df['rectal_exam_feces'].fillna('Unknown', inplace=True)
train_df['abdomen'].fillna('Unknown', inplace=True)

temp_of_extremities,
peripheral_pulse, 
mucous_membrane, 
capillary_refill_time,  
pain, 
peristalsis, 
abdominal_distention,  
nasogastric_tube,  
 nasogastric_reflux,  
 abdomo_appearance 
 

In [ ]:
train_df['temp_of_extremities'].fillna(train_df['temp_of_extremities'].mode()[0], inplace=True)
train_df['peripheral_pulse'].fillna('Unknown', inplace=True)

train_df['mucous_membrane'].fillna(train_df['mucous_membrane'].mode()[0], inplace=True)
train_df['capillary_refill_time'].fillna('Unknown', inplace=True)

train_df['capillary_refill_time'].fillna(train_df['capillary_refill_time'].mode()[0], inplace=True)
train_df['pain'].fillna('Unknown', inplace=True)

train_df['peristalsis'].fillna(train_df['peristalsis'].mode()[0], inplace=True)
train_df['abdominal_distention'].fillna('Unknown', inplace=True)

train_df['nasogastric_tube'].fillna(train_df['nasogastric_tube'].mode()[0], inplace=True)
train_df['nasogastric_reflux'].fillna('Unknown', inplace=True)

train_df['nasogastric_reflux'].fillna(train_df['nasogastric_reflux'].mode()[0], inplace=True)
train_df['abdomo_appearance'].fillna('Unknown', inplace=True)



In [ ]:
print(train_df.isnull().sum())

## Test Data

In [ ]:
test_df.shape

Test datset has 824 rows and 28 columns.

In [ ]:
test_df.head()

In [ ]:
test_df.info()

Test Dataset has columns with numerical and categorical datatypes. Next will explore missinf value and check the datatype. 

### Checking dublicates and missing value: 

In [ ]:
# Let's check for duplicate values in the data
test_df.duplicated().sum()

Test Data has 0 dublicates.

In [ ]:
# Let's check for missing values in the data
round(test_df.isnull().sum() / test_df.isnull().count() * 100, 2)

Test data has columns with missing value, %: 
- temp_of_extremities       4.25
- peripheral_pulse          5.70
- mucous_membrane           1.58
- capillary_refill_time     0.73
- pain                      3.52
- peristalsis               2.31
- abdominal_distention      2.67
- nasogastric_tube          7.77
- nasogastric_reflux        1.70
- rectal_exam_feces        15.17
- abdomen                  18.69
- abdomo_appearance         3.76

In [ ]:
#Checking Missing Values Pattern in Dataframe
import missingno as mn

null_cols = [i for i in test_df.columns if test_df[i].isnull().any()]
print(null_cols)
mn.matrix(test_df[null_cols])

In [ ]:
# Let's check the number of unique values in each column
test_df.nunique()

#### Data Pre-processing Test_df

In [ ]:
# ID column consists of uniques ID and hence will not add value to the modeling

saved_ids_test = test_df['id'].copy()

test_df.drop(columns="id", inplace=True)

In [ ]:
# ID column consists of uniques hospital_number and hence will not add value to the modeling
test_df.drop(columns="hospital_number", inplace=True)

 ### Impute Missing Values in Test Dataset:

#### - Numeric Columns (float64/int64):

In [ ]:
# Using the median of the training dataset to fill missing values in the test dataset
test_df['rectal_temp'].fillna(train_df['rectal_temp'].median(), inplace=True)
test_df['pulse'].fillna(train_df['pulse'].median(), inplace=True)
test_df['respiratory_rate'].fillna(train_df['respiratory_rate'].median(), inplace=True)
test_df['nasogastric_reflux_ph'].fillna(train_df['nasogastric_reflux_ph'].median(), inplace=True)
test_df['packed_cell_volume'].fillna(train_df['packed_cell_volume'].median(), inplace=True)
test_df['total_protein'].fillna(train_df['total_protein'].median(), inplace=True)
test_df['abdomo_protein'].fillna(train_df['abdomo_protein'].median(), inplace=True)


#### -Categorical Columns (object):

In [ ]:
# Using the mode of the training dataset or 'Unknown' to fill missing values in the test dataset
test_df['temp_of_extremities'].fillna(train_df['temp_of_extremities'].mode()[0], inplace=True)
test_df['peripheral_pulse'].fillna('Unknown', inplace=True)
test_df['mucous_membrane'].fillna(train_df['mucous_membrane'].mode()[0], inplace=True)
test_df['capillary_refill_time'].fillna(train_df['capillary_refill_time'].mode()[0], inplace=True)
test_df['pain'].fillna(train_df['pain'].mode()[0], inplace=True)
test_df['peristalsis'].fillna(train_df['peristalsis'].mode()[0], inplace=True)
test_df['abdominal_distention'].fillna(train_df['abdominal_distention'].mode()[0], inplace=True)
test_df['nasogastric_tube'].fillna(train_df['nasogastric_tube'].mode()[0], inplace=True)
test_df['nasogastric_reflux'].fillna(train_df['nasogastric_reflux'].mode()[0], inplace=True)
test_df['abdomo_appearance'].fillna(train_df['abdomo_appearance'].mode()[0], inplace=True)


#### - Special Columns:

In [ ]:
# Using 'Unknown' to fill missing values for specific columns
test_df['rectal_exam_feces'].fillna('Unknown', inplace=True)
test_df['abdomen'].fillna('Unknown', inplace=True)


In [ ]:
print(test_df.isnull().sum())


### Modeling

#### Data Preparation

In [ ]:
pd.set_option('display.max_columns', None) 

In [ ]:
train_df.head(10)

In [ ]:
# Define the mappings
mappings = {
    'surgery':{'yes':24, 'no': 25},
    'age': {'adult': 1, 'young': 0},
    'temp_of_extremities': {'cool': 21, 'normal': 23, 'cold': 22, 'warm': 24},
    'peripheral_pulse': {'reduced': 31, 'normal': 32, 'Unknown': 34, 'absent': 35, 'increased': 33},
    'mucous_membrane': {'pale_pink': 41, 'normal_pink': 42, 'pale_cyanotic': 43, 'bright_red': 44, 'dark_cyanotic': 45, 'bright_pink': 46},
    'capillary_refill_time': {'less_3_sec': 51, 'more_3_sec': 52, 'Unknown': 53, '3': 55},
    'pain': {'depressed': 61, 'mild_pain': 62, 'severe_pain': 63, 'extreme_pain': 64, 'alert': 65, 'Unknown': 66, 'slight': 67},
    'peristalsis': {'hypomotile': 71, 'absent': 72, 'normal': 73, 'hypermotile': 75, 'distend_small': 78},
    'abdominal_distention': {'moderate': 81, 'slight': 82, 'none': 83, 'severe': 85, 'Unknown': 88},
    'nasogastric_tube': {'slight': 91, 'none': 92, 'significant': 93},
    'nasogastric_reflux': {'more_1_liter': 10, 'none': 20, 'less_1_liter': 30, 'Unknown': 40, 'slight': 50},
    'rectal_exam_feces': {'absent': 112, 'decreased': 112, 'normal': 113, 'Unknown': 114, 'increased': 115, 'serosanguious': 116},
    'abdomen': {'distend_small': 121, 'distend_large': 123, 'Unknown': 124, 'normal': 125, 'firm': 126, 'other': 127},
    'abdomo_appearance': {'serosanguious': 131, 'cloudy': 132, 'clear': 133, 'Unknown': 134},
    'surgical_lesion': {'yes': 145, 'no': 165},
    'cp_data': {'yes': 134, 'no': 334},
    'outcome': {'lived': 1, 'died': 2, 'euthanized': 3}
}

# Apply the mappings to the dataframe
for column, mapping in mappings.items():
    train_df[column] = train_df[column].map(mapping)


In [ ]:
train_df

In [ ]:
train_df.info()

In [ ]:
# Define the mappings
mappings = {
    'surgery':{'yes':24, 'no': 25},
    'age': {'adult': 1, 'young': 0},
    'temp_of_extremities': {'cool': 21, 'normal': 23, 'cold': 22, 'warm': 24},
    'peripheral_pulse': {'reduced': 31, 'normal': 32, 'Unknown': 34, 'absent': 35, 'increased': 33},
    'mucous_membrane': {'pale_pink': 41, 'normal_pink': 42, 'pale_cyanotic': 43, 'bright_red': 44, 'dark_cyanotic': 45, 'bright_pink': 46},
    'capillary_refill_time': {'less_3_sec': 51, 'more_3_sec': 52, 'Unknown': 53, '3': 55},
    'pain': {'depressed': 61, 'mild_pain': 62, 'severe_pain': 63, 'extreme_pain': 64, 'alert': 65, 'Unknown': 66, 'slight': 67},
    'peristalsis': {'hypomotile': 71, 'absent': 72, 'normal': 73, 'hypermotile': 75, 'distend_small': 78},
    'abdominal_distention': {'moderate': 81, 'slight': 82, 'none': 83, 'severe': 85, 'Unknown': 88},
    'nasogastric_tube': {'slight': 91, 'none': 92, 'significant': 93},
    'nasogastric_reflux': {'more_1_liter': 10, 'none': 20, 'less_1_liter': 30, 'Unknown': 40, 'slight': 50},
    'rectal_exam_feces': {'absent': 112, 'decreased': 112, 'normal': 113, 'Unknown': 114, 'increased': 115, 'serosanguious': 116},
    'abdomen': {'distend_small': 121, 'distend_large': 123, 'Unknown': 124, 'normal': 125, 'firm': 126, 'other': 127},
    'abdomo_appearance': {'serosanguious': 131, 'cloudy': 132, 'clear': 133, 'Unknown': 134},
    'surgical_lesion': {'yes': 145, 'no': 165},
    'cp_data': {'yes': 134, 'no': 334}
}

# Apply the mappings to the dataframe
for column, mapping in mappings.items():
    test_df[column] = test_df[column].map(mapping)


In [ ]:
test_df['pain'].fillna(test_df['pain'].mean(), inplace=True)


In [ ]:
# Load the sample submission file
sample_submission_filepath = "/kaggle/input/playground-series-s3e22/sample_submission.csv"
sample_submission = pd.read_csv(sample_submission_filepath)

# Update the target column with your predictions
#sample_submission['target_column_name'] = y_pred_test

# Save the submission file
#sample_submission.to_csv('submission.csv', index=False)

In [ ]:
sample_submission.shape

In [ ]:
sample_submission.head()

In [ ]:
sample_submission.isnull().sum()

No missing values.

In [ ]:
sample_submission.nunique()

In [ ]:
# Define the mappings
mappings = {
    'outcome': {'lived': 1, 'died': 2, 'euthanized': 3}
}

# Apply the mappings to the dataframe
for column, mapping in mappings.items():
    sample_submission[column] = sample_submission[column].map(mapping)

In [ ]:
sample_submission.info()

In [ ]:
y_test = sample_submission['outcome']

 #### Splitting the Training Data

In [ ]:
from sklearn.model_selection import train_test_split

X = train_df.drop(columns='outcome')
y = train_df['outcome'].astype(int)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)


### Scalling Data: 

In [ ]:
# Scaling the data
sc=StandardScaler()

# Fit_transform on train data
X_train_scaled=sc.fit_transform(X_train)
X_train_scaled=pd.DataFrame(X_train_scaled, columns=X.columns)



In [ ]:
# Transform on test data
X_val_scaled=sc.transform(X_val)
X_val_scaled=pd.DataFrame(X_val_scaled, columns=X.columns)

In [ ]:
X_test_scaled = sc.transform(test_df)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=test_df.columns)

### Model evaluation criterion

**Goal: Use the above medical indicators to predict the health outcomes of horses.**  

Given the goal of predicting the health outcomes of horses, the two types of wrong predictions your model can make are:

1. **False Positive (FP)**: Predicting that a horse will experience a certain health outcome when it will not. For example, predicting that a horse will "die" when it actually "lives."
2. **False Negative (FN)**: Predicting that a horse will not experience a certain health outcome when it will. For example, predicting that a horse will "live" when it actually "dies."

The implications of these errors in the context of predicting health outcomes for horses are:

- A **False Positive** might lead to unnecessary interventions, treatments, or even euthanasia if the prediction is severe. It can also lead to unnecessary costs and stress.
  
- A **False Negative** can be more dangerous. If the model predicts that a horse is going to be fine ("live") when it's actually at risk ("die" or "euthanized"), it might not receive the necessary medical attention in time, leading to potential harm or death.

In many medical or health-related contexts, reducing **False Negatives** is often more critical because of the potential harm that can arise from missing a crucial diagnosis or intervention. 

- Evaluation metric should give more penalty to the type of error you deem more critical.  Given that using the **F1-Score**, it takes both precision and recall into account, balancing the trade-off between **False Positives** and **False Negatives**.

In [ ]:
# Function to print the classification report and get confusion matrix in a proper format

def metrics_score(actual, predicted):
    print(classification_report(actual, predicted))
    
    cm = confusion_matrix(actual, predicted)
    
    plt.figure(figsize = (8, 5))
    
    sns.heatmap(cm, annot = True,  fmt = '.2f', xticklabels = ['Not Converted', 'Converted'], yticklabels = ['Not Converted', 'Converted'])
    
    plt.ylabel('Actual')
    
    plt.xlabel('Predicted')
    
    plt.show()

## Building a Decision Tree model

In [ ]:
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(X_train_scaled, y_train)

In [ ]:
# Checking performance on the training dataset
y_train_pred_dt = dt_classifier.predict(X_train_scaled)
score = metrics_score(y_train, y_train_pred_dt)

In [ ]:
# Checking performance on the validation dataset
y_val_pred_dt = dt_classifier.predict(X_val_scaled)
score = metrics_score(y_val, y_val_pred_dt)

In [ ]:
# Checking performance on the test dataset
y_test_pred_dt = dt_classifier.predict(X_test_scaled)
score = metrics_score(y_test, y_test_pred_dt)

In [ ]:
# Importance of features in the tree building

Imp_f_d_tree = pd.DataFrame(dt_classifier.feature_importances_, columns = ["Imp"], index = X_train.columns).sort_values(by = 'Imp', ascending = False)
Imp_f_d_tree 

In [ ]:
# Plot the feature importance

importances = dt_classifier.feature_importances_
columns = X.columns
importance_df = pd.DataFrame(importances, index = columns, columns = ['Importance']).sort_values(by = 'Importance', ascending = False)
plt.figure(figsize = (8, 8))
sns.barplot(data = importance_df, x = importance_df.Importance, y = importance_df.index);

### Visualizing the Decision Tree

In [ ]:
features = list(X.columns)

plt.figure(figsize = (30, 20))

tree.plot_tree(dt_classifier, max_depth = 6, feature_names = features, filled = True, fontsize = 12, node_ids = True, class_names = True)

plt.show()

### Prune Tree Model

In [ ]:
# Get unique classes in y_train
unique_classes = np.unique(y_train)

# If you have specific weights you want to assign, you can do so here
class_weights = {cls: weight for cls, weight in zip(unique_classes, [0.3, 0.7, 0.7])}

# Update the class_weight parameter in the classifier
d_tree_tuned = DecisionTreeClassifier(random_state=7, class_weight=class_weights)

# Continue with the rest of your code
parameters = {'max_depth': np.arange(2, 10), 
              'criterion': ['gini', 'entropy'],
              'min_samples_leaf': [5, 10, 20, 25]
             }

scorer = metrics.make_scorer(recall_score, pos_label=1)

grid_obj = GridSearchCV(d_tree_tuned, parameters, scoring=scorer, cv=5)
grid_obj = grid_obj.fit(X_train_scaled, y_train)

d_tree_tuned = grid_obj.best_estimator_
d_tree_tuned.fit(X_train_scaled, y_train)


In [ ]:
# Checking performance on the training data

y_train_pred_dt2 = d_tree_tuned.predict(X_train_scaled)

score_train = metrics_score(y_train, y_train_pred_dt2)

In [ ]:
# Checking performance on the validation data

y_val_pred_dt2 = d_tree_tuned.predict(X_val_scaled)

score_test = metrics_score(y_val, y_val_pred_dt2)

In [ ]:
# Checking performance on the test dataset
y_test_pred_dt = d_tree_tuned.predict(X_test_scaled)
score = metrics_score(y_test, y_test_pred_dt)

#### Let's visualize the tuned decision tree and observe the decision rules:

In [ ]:
features = list(X.columns)

plt.figure(figsize = (30, 20))

tree.plot_tree(d_tree_tuned, max_depth = 6, feature_names = features, filled = True, fontsize = 12, node_ids = True, class_names = True)

plt.show()

In [ ]:
# Importance of features in the tree building

Imp_f_d_tuned = pd.DataFrame(d_tree_tuned.feature_importances_, columns = ["Imp"], index = X_train.columns).sort_values(by = 'Imp', ascending = False)
Imp_f_d_tuned

In [ ]:
# Plot the feature importance

importances = d_tree_tuned.feature_importances_
columns = X.columns
importance_df = pd.DataFrame(importances, index = columns, columns = ['Importance']).sort_values(by = 'Importance', ascending = False)
plt.figure(figsize = (8, 8))
sns.barplot(data = importance_df, x = importance_df.Importance, y = importance_df.index);

### Building a Random Forest model

In [ ]:
#Create an instance of the RandomForestClassifier
rf_estimator = RandomForestClassifier(random_state=42, n_estimators=100)

#Fit the model on the training data
rf_estimator.fit(X_train_scaled, y_train)

In [ ]:
# Making predictions on the test data
y_test_pred_rf = rf_estimator.predict(X_test_scaled)

In [ ]:
#Predicting the training data labels
y_pred_train3 = rf_estimator.predict(X_train_scaled)

score_train_rf = metrics_score(y_train, y_pred_train3)

In [ ]:
#Predicting the validation data labels
y_pred_val3 = rf_estimator.predict(X_val_scaled)

score_test_rf = metrics_score(y_val, y_pred_val3)

In [ ]:
#Predicting the test data labels
y_test_val3 = rf_estimator.predict(X_test_scaled)

score_test_rf = metrics_score(y_test, y_test_val3)

### Random Forest Classifier - Hyperparameter Tuning

In [ ]:
# Choose the type of classifier
rf_estimator_tuned = RandomForestClassifier(criterion = "entropy", random_state = 7)

# Grid of parameters to choose from
parameters = {
    "n_estimators": [110, 120],
    "max_depth": [6, 7],
    "min_samples_leaf": [20, 25],
    "max_features": [0.8, 0.9],
    "max_samples": [0.9, 1],
    "class_weight": ["balanced", {0: 0.3, 1: 0.7}]
}

# Type of scoring used to compare parameter combinations
scorer = metrics.make_scorer(recall_score, pos_label = 1)

# Run the grid search on the training data
grid_obj = GridSearchCV(rf_estimator_tuned, parameters, scoring=scorer, cv=5)
grid_obj = grid_obj.fit(X_train_scaled, y_train)

# Save the best estimator to variable rf_estimator_tuned
rf_estimator_tuned = grid_obj.best_estimator_

# Display the best parameters found by the grid search
print('Best parameters found: ', grid_obj.best_params_)

In [ ]:
rf_estimator_tuned.fit(X_train_scaled, y_train)

In [ ]:
# Checking performance on the training data

y_pred_train_rft = rf_estimator_tuned.predict(X_train_scaled)

score_train_rft = metrics_score(y_train, y_pred_train_rft)

In [ ]:
# Checking performance on the test data

y_pred_val_rft = rf_estimator_tuned.predict(X_val_scaled)

score_test_rft = metrics_score(y_val, y_pred_val_rft)

In [ ]:
# Checking performance on the test data

y_pred_test_rft = rf_estimator_tuned.predict(X_test_scaled)

score_test_rft = metrics_score(y_test, y_pred_test_rft)

In [ ]:
# Importance of features in the tree building

Imp_f_fr_tuned = pd.DataFrame(rf_estimator_tuned.feature_importances_, columns = ["Imp"], index = X_train.columns).sort_values(by = 'Imp', ascending = False)
Imp_f_fr_tuned 

In [ ]:
importances = rf_estimator_tuned.feature_importances_ 
columns = X.columns
importance_df = pd.DataFrame(importances, index = columns, columns = ['Importance']).sort_values(by = 'Importance', ascending = False)
plt.figure(figsize = (8, 8))
sns.barplot(data = importance_df, x = importance_df.Importance, y = importance_df.index);